In [ ]:
from datetime import datetime

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import metpy.calc as mpcalc
from metpy.interpolate import interpolate_to_grid
from metpy.io import add_station_lat_lon
from metpy.plots import StationPlot
from metpy.units import units
import numpy as np
from siphon.simplewebservice.iastate import IAStateUpperAir 
import xarray as xr

# Grid Interpolation from Point Data
There are times when we have point data but want to make a gridded analysis to be able to have the computer draw contours. While there are many gridded datasets available, sometimes it is best to only use the data you have to create your analysis, especially if you have extra data that were not used in the construction of other gridded products, or you are looking to do a mesoscale analysis over a smaller region. In this notebook we'll use some upperair geopotential height observations over the CONUS available from Iowa State data archives to demonstrate the capabilities in MetPy for gridding up point observations.

## Upperair Observations
There are a couple of different places to retrieve upperair observations from sounding archives. Previously we have used the Wyoming archive to obtain individual stations vertical profiles. Here we will use Siphon and the Iowa State archive to be able to get all of the data available for a given date/time.

In [ ]:
# Set date to get observations from
date = datetime(1978, 1, 26, 12)

# Use Siphon Simple Web Service to get all UPA data from IA State archive
df = IAStateUpperAir.request_all_data(date)

In [ ]:
df

## Pre-process Observations
Not all of the needed infromation comes along with our dataset, we need to add station lat/lon information, drop any observations where we don't have any station lat/lon information added and we'll just look at 500-hPa observations in this example.

In [ ]:
# Add station lat/lon locations to DataFrame
df = add_station_lat_lon(df)

# Drop any station we don't have a lat/lon for


# Select only 500 hPa data


## Plotting Point Observations with MetPy

MetPy offers the ability to plot surface and upperair observations using a station model. Here we are not going to plot all of the potential variables for an upper-level station model, but demonstrate how to plot the 500-hPa Geopotential Height values. Just like there was a specific plotting class for our SkewT digrams, here we have a station plot class that we'll create using the lat/lon points of the observations we wish to plot. See the link to the documentation below for all of the detailed information.

StationPlot: https://unidata.github.io/MetPy/latest/api/generated/metpy.plots.StationPlot.html

Once we set up the station object, we can add observations around the station model using cardinal directions ('N', 'S', 'E', 'W', 'NE', 'SE', 'SW', 'NW', etc.) and a center location ('C'). And we are plotting a numeric parameter, so we'll use the `plot_parameter` method off of our station object.

In [ ]:
plt.figure(1, figsize=(15, 13))
ax = plt.subplot(111, projection=ccrs.LambertConformal(central_longitude=-100, central_latitude=40))
ax.set_extent([-125, -65, 25, 55], ccrs.PlateCarree())

# Set up station object using lat/lon info from our Obs


# Plot the Geopotnential Height values at the center location



# Add Geographic Features
ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax.add_feature(cfeature.BORDERS.with_scale('50m'))
ax.add_feature(cfeature.STATES.with_scale('50m'))

# Add titles
plt.title(f'500-hPa Observations', loc='left')
plt.title(f'Valid: {date}', loc='right')

plt.show()

## Creating a Grid Interploation
With the above created image we could draw contours of geopotential height by hand to create an interpolation or we can generate a grid, have the computer do an interpolation of the observations, and then have the computer draw some contours. So let's try that out...

*But what types of interpolation could we use?*

There are a number of different interpolation methods that are available in the `interpolate_to_grid` function available from MetPy. Some of them are historic to meteorology (e.g., `'cressman'`, `'barnes'`) and others are common interpolation methods from mathematics/statistics (e.g., `'linear'`, '`cubic'`, `'rbf'`). Depending on which type of interpolation method you use, there might be additional parameters that can be set to refine the output from the interpolation.

*What else do I need to set?*

The other key element to set at minimum is the `hres` parameter, which dictates the spacing between grid points in the gridded dataset. This parameter should be set according to the units of your x and y values. If using lat/lon directly, then the units are degrees. If you are in some cartesian or projected space, the units are likely to be in meters or kilometers.

In [ ]:
# Generate interpolated grid
gridx, gridy, grid_hght = interpolate_to_grid()

# Smooth the grid using a MetPy function
grid_hght_smooth = mpcalc.smooth_n_point(grid_hght, 9, 3)

In [ ]:
grid_hght

## Add contours to Observations
Use the new interpolated grid to plot contours using Matplotlib and add to the map of our observations.

In [ ]:
plt.figure(1, figsize=(15, 13))
ax = plt.subplot(111, projection=ccrs.LambertConformal(central_longitude=-100, central_latitude=40))
ax.set_extent([-125, -65, 25, 55], ccrs.PlateCarree())

station = StationPlot(ax, data_500.longitude, data_500.latitude, transform=ccrs.PlateCarree(), clip_on=True)

station.plot_parameter('C', data_500.height)
station.plot_text('S', data_500.station)

# Add contours of interpolated geopotential heights
cs = 

plt.clabel(cs)

# Add Geographic Features
ax.add_feature(cfeature.COASTLINE.with_scale('50m'), edgecolor='grey')
ax.add_feature(cfeature.BORDERS.with_scale('50m'), edgecolor='grey')
ax.add_feature(cfeature.STATES.with_scale('50m'), edgecolor='grey')

# Add titles
plt.title(f'500-hPa Observations', loc='left')
plt.title(f'Valid: {date}', loc='right')


plt.show()

# Bad Observations
When conducting our interpolation on raw observations, there could be bad (or misplaced) data points that greatly impact the interpolation and resulting contour analysis. Using data from 26 January 1978 at 12 UTC there are two observations that don't seem to fit.
1. Over central Misouri there is a misplaced station 'KVER' that should be removed
2. In Arizona, the Yuma sounding 'KYUX' seems to have a wrong value for the 500-hPa geopotential height

So let's work to remove those data points and re-do our analysis.

In [ ]:
# QC Data
data_500

In [ ]:
#Apply boolean conditionals to remove the two offending points from the data frame
data_500 = 

In [ ]:
#Redo the grid
gridx, gridy, grid_hght = interpolate_to_grid(data_500.longitude, data_500.latitude, data_500.height,
                                              interp_type='rbf', hres=1)
grid_hght_smooth = mpcalc.smooth_n_point(grid_hght, 9, 3)

In [ ]:
plt.figure(2, figsize=(15, 13))
ax = plt.subplot(111, projection=ccrs.LambertConformal(central_longitude=-100, central_latitude=40))
ax.set_extent([-125, -65, 25, 55], ccrs.PlateCarree())



# Add Geographic Features
ax.add_feature(cfeature.COASTLINE.with_scale('50m'), edgecolor='grey')
ax.add_feature(cfeature.BORDERS.with_scale('50m'), edgecolor='grey')
ax.add_feature(cfeature.STATES.with_scale('50m'), edgecolor='grey')

# Add titles
plt.title(f'500-hPa Observations', loc='left')
plt.title(f'Valid: {date}', loc='right')


plt.show()

## Plot UPA Station Model with Contours
Here we plot the full upperair station model with our interpolated grid contours. Looks like our interpolation did a good job where we have observations of representing the geopotential heights as the winds indicate the geostrophic balance is obtained.

In [ ]:
plt.figure(3, figsize=(15, 13))
ax = plt.subplot(111, projection=ccrs.LambertConformal(central_longitude=-100, central_latitude=40))
ax.set_extent([-125, -65, 25, 55], ccrs.PlateCarree())

station = StationPlot(ax, data_500.longitude, data_500.latitude, fontsize=8, spacing=11, transform=ccrs.PlateCarree(), clip_on=True)

# Add all station data around obs point




# Add Geographic Features
ax.add_feature(cfeature.COASTLINE.with_scale('50m'), edgecolor='grey')
ax.add_feature(cfeature.BORDERS.with_scale('50m'), edgecolor='grey')
ax.add_feature(cfeature.STATES.with_scale('50m'), edgecolor='grey')

# Add titles
plt.title(f'500-hPa Observations', loc='left')
plt.title(f'Valid: {date}', loc='right')


plt.show()